In [1]:
import hub
import torch
import numpy as np
from hub.schema import Image, ClassLabel
from hub.utils import Timer
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensor

In [2]:
ds = hub.load("activeloop/mnist")

### Converting as is

In [3]:
mnist_torch = ds.to_pytorch()

In [4]:
print(mnist_torch[0].keys())
print(mnist_torch[0]['image'].size())

dict_keys(['image', 'label'])
torch.Size([28, 28, 1])


Each sample of dataset, converted to pytorch with default parameters, is a dictionary with keys 'image' and 'label', shape of images is [28, 28, 1] -> H, W, C. Note, that this is not the order used in pytorch (should be C x H x W). We will try to output list and tuple

In [5]:
mnist_torch = ds.to_pytorch(output_type=list)
print(mnist_torch[0])

[tensor([[[  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
   

In [6]:
mnist_torch = ds.to_pytorch(output_type=tuple)
print(mnist_torch[0])

(tensor([[[  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0]],

        [[  0],
         [  0],
         [  0],
         [  0],
         [  0],
         [  0],
   

### Converting with Transforms  - TorchVision

Let's define torchvision transformations as a function. The function should work with initial dataset structure.

In [7]:
torchvision_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)), 
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.1307,), # mean value for mnist dataset
        std=(0.3081,), # mean value for mnist dataset
    )
])

def transform(data):
    img = torchvision_transform(data['image'])
    label = torch.tensor(data['label'])
    return img, label

We use `inplace=False` since we want to tranform original numpy arrays. Note, that now transform function defines output type

In [8]:
mnist_torch = ds.to_pytorch(Transform=transform, inplace=False)

print(mnist_torch[0])

(tensor([[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         ...,
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]]), tensor(2))


### Converting with Albumentations  - TorchVision

In [9]:
albumentations_transform = A.Compose([
    A.Resize(256, 256), 
    A.RandomCrop(224, 224),
    A.HorizontalFlip(),
    A.Normalize(
        mean=(0.1307,),
        std=(0.3081,),
    ),
    ToTensor()
])

In [10]:
def transform(data):
    augmented = albumentations_transform(image=data['image'])
    img = augmented['image']
    label = torch.tensor(data['label'])
    return img, label

In [11]:
mnist_torch = ds.to_pytorch(Transform=transform, inplace=False)

In [12]:
print(mnist_torch[0])

(tensor([[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         ...,
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
         [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]]), tensor(2))
